In [1]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Read all result file addresses, change to the path where you saved the files here
path = r'D:\Projects\Research\SolarStudy\ExcelFiles'                     
all_files = glob.glob(os.path.join(path, "*.xlsx")) # OS independent

In [3]:
#Extract GH input parameters (seed numbers) from the file names and save into a pandas dataframe
#Note: use these to run GH definition without the simulation and generate geometrical properties of 
#each model, e.g. heights, widths, distances, etc.

N_rows = len(all_files)
N_cols = 4 #number of input parameters from GH file

Inputs = pd.DataFrame(np.zeros((N_rows, N_cols)), columns=['RSBS', 'RSBH', 'RSRO', 'LT'])
for i in range(0, len(all_files)):
    interim = str.split(all_files[i], sep='\\')
    parameters = str.split(interim[5], sep='_')
    a = len(parameters)-1
    parameters_2 = str.split(parameters[a], sep='.')
    Inputs['RSBS'][i] = parameters[1]
    Inputs['RSBH'][i] = parameters[3]
    Inputs['RSRO'][i] = parameters[5]
    Inputs['LT'][i] = parameters_2[0]

In [7]:
Inputs.head()

,RSBS,RSBH,RSRO,LT
0,15.0,153.0,50.0,1.0
1,15.0,153.0,51.0,0.0
2,15.0,153.0,51.0,1.0
3,15.0,153.0,52.0,0.0
4,15.0,153.0,52.0,1.0


In [5]:
#Extract the actual results from the excel files. Only the last 11595 are extracted, 
#representing points of the street network mesh
Data = np.empty([0,11595])

for f in all_files:
    temp = pd.read_excel(f, header=None, use_cols = "A")
    temp = temp[-11595:]
    Data = np.append(Data, temp).reshape(-1,11595)

In [6]:
print(Data.shape)
print(Inputs.shape)

(3260, 11595)
(3260, 4)


In [8]:
#Save Inputs and Data file for later use
np.save('D:\Projects\Research\SolarStudy\Inputs.npy', Inputs)
np.save('D:\Projects\Research\SolarStudy\Data.npy', Data)

In [ ]:
#Load Inputs and Data file at a later stage if necessary
Inputs = np.load('D:\Projects\Research\SolarStudy\Inputs.npy')
Data = np.load('D:\Projects\Research\SolarStudy\Data.npy')

In [9]:
#Generate basic statistics for our results, which will help us 'split' them into classes
AverageRadiation = np.empty([3260,1])
MaxRadiation = np.empty([3260,1])
MinRadiation = np.empty([3260,1])
MedianRadiation = np.empty([3260,1])

for i in range(0, len(Data)):
    AverageRadiation[i] = np.average(Data[i])
    MaxRadiation[i] = np.max(Data[i])
    MinRadiation[i] = np.min(Data[i])
    MedianRadiation[i] = MaxRadiation[i] - MinRadiation[i]

In [10]:
#Use maximum radiation to create threshold values which will classify areas according 
#to the percentage of maximum radiation falling on them, for each iteration
area_10_threshold = np.empty([3260,1])
area_20_threshold = np.empty([3260,1])
area_30_threshold = np.empty([3260,1])
area_40_threshold = np.empty([3260,1])
area_50_threshold = np.empty([3260,1])
area_60_threshold = np.empty([3260,1])
area_70_threshold = np.empty([3260,1])
area_80_threshold = np.empty([3260,1])
area_90_threshold = np.empty([3260,1])
area_100_threshold = np.empty([3260,1])

for i in range(0, len(MaxRadiation)):
    area_10_threshold[i][0] = 0.1 * MaxRadiation[i][0]
    area_20_threshold[i][0] = 0.2 * MaxRadiation[i][0]
    area_30_threshold[i][0] = 0.3 * MaxRadiation[i][0]
    area_40_threshold[i][0] = 0.4 * MaxRadiation[i][0]
    area_50_threshold[i][0] = 0.5 * MaxRadiation[i][0]
    area_60_threshold[i][0] = 0.6 * MaxRadiation[i][0]
    area_70_threshold[i][0] = 0.7 * MaxRadiation[i][0]
    area_80_threshold[i][0] = 0.8 * MaxRadiation[i][0]
    area_90_threshold[i][0] = 0.9 * MaxRadiation[i][0]
    area_100_threshold[i][0] = 1 * MaxRadiation[i][0]
    
#Calculate total area within each class, for all iterations
area_0 = np.zeros([3260,11595])
area_10 = np.zeros([3260,11595])
area_20 = np.zeros([3260,11595])
area_30 = np.zeros([3260,11595])
area_40 = np.zeros([3260,11595])
area_50 = np.zeros([3260,11595])
area_60 = np.zeros([3260,11595])
area_70 = np.zeros([3260,11595])
area_80 = np.zeros([3260,11595])
area_90 = np.zeros([3260,11595])
area_100 = np.zeros([3260,11595])

for i in range(0, len(Data)):
    for k in range(0, len(Data[0])):
        if(Data[i][k] < area_10_threshold[i]):
            area_0[i][k] += 1
        elif(Data[i][k] >= area_10_threshold[i] and Data[i][k] < area_20_threshold[i][0]):
            area_10[i][k] += 1
        elif(Data[i][k] >= area_20_threshold[i] and Data[i][k] < area_30_threshold[i][0]):
            area_20[i][k] += 1
        elif(Data[i][k] >= area_30_threshold[i] and Data[i][k] < area_40_threshold[i][0]):
            area_30[i][k] += 1
        elif(Data[i][k] >= area_40_threshold[i] and Data[i][k] < area_50_threshold[i][0]):
            area_40[i][k] += 1
        elif(Data[i][k] >= area_50_threshold[i] and Data[i][k] < area_60_threshold[i][0]):
            area_50[i][k] += 1
        elif(Data[i][k] >= area_60_threshold[i] and Data[i][k] < area_70_threshold[i][0]):
            area_60[i][k] += 1
        elif(Data[i][k] >= area_70_threshold[i] and Data[i][k] < area_80_threshold[i][0]):
            area_70[i][k] += 1
        elif(Data[i][k] >= area_80_threshold[i] and Data[i][k] < area_90_threshold[i][0]):
            area_80[i][k] += 1
        elif(Data[i][k] >= area_90_threshold[i] and Data[i][k] < area_100_threshold[i][0]):
            area_90[i][k] += 1
        else:
            area_100[i][k] += 1

In [11]:
#Summing up areas per category and case
area_0_sums = np.zeros([3260,1])
area_10_sums = np.zeros([3260,1])
area_20_sums = np.zeros([3260,1])
area_30_sums = np.zeros([3260,1])
area_40_sums = np.zeros([3260,1])
area_50_sums = np.zeros([3260,1])
area_60_sums = np.zeros([3260,1])
area_70_sums = np.zeros([3260,1])
area_80_sums = np.zeros([3260,1])
area_90_sums = np.zeros([3260,1])
area_100_sums = np.zeros([3260,1])

for i in range(0, len(area_10)):
    area_0_sums[i] = np.sum(area_0[i])
    area_10_sums[i] = np.sum(area_10[i])
    area_20_sums[i] = np.sum(area_20[i])
    area_30_sums[i] = np.sum(area_30[i])
    area_40_sums[i] = np.sum(area_40[i])
    area_50_sums[i] = np.sum(area_50[i])
    area_60_sums[i] = np.sum(area_60[i])
    area_70_sums[i] = np.sum(area_70[i])
    area_80_sums[i] = np.sum(area_80[i])
    area_90_sums[i] = np.sum(area_90[i])
    area_100_sums[i] = np.sum(area_100[i])
    
#Concatanate into one area sum array
Area_sums = np.hstack((area_0_sums, area_10_sums, area_20_sums, area_30_sums, area_40_sums,
                      area_50_sums, area_60_sums, area_70_sums, area_80_sums, area_90_sums,
                      area_100_sums))

#Create the total area dataframe for each iteration and generate the class (as the category with maximum area)
Total_areas = pd.DataFrame(data=Area_sums, columns=['0', '10', '20', '30', '40', '50', '60', '70', '80',
                                                   '90', '100'])

Total_areas = (Total_areas / len(Data[0])) * 100 #Turn areas into percentage of area under each category

Total_areas['Class'] = Total_areas.idxmax(axis=1)

#Number of classes
print('Total number of classes identified:', len(Total_areas['Class'].unique()))
print('Classes are:', Total_areas['Class'].unique())
Total_areas.head()

Total number of classes identified: 6
Classes are: ['60' '50' '70' '80' '40' '30']


,0,10,20,30,40,50,60,70,80,90,100,Class
0,0.0,0.0,0.0,0.000000,0.457094,9.969815,30.245796,28.098318,25.373006,5.847348,0.008624,60
1,0.0,0.0,0.0,1.026304,7.951703,26.002587,24.795170,17.688659,9.383355,13.074601,0.077620,50
2,0.0,0.0,0.0,0.000000,0.871065,9.374730,29.417853,28.150065,24.984907,7.184131,0.017249,60
3,0.0,0.0,0.0,0.551962,8.210436,25.329884,26.571798,17.636912,8.684778,12.919362,0.094868,60
4,0.0,0.0,0.0,0.000000,0.448469,9.391979,29.771453,30.228547,24.691677,5.424752,0.043122,70


In [12]:
#Summing category results by pair to create classes with higher contribution to the overall result.
Total_areas_aggregate = np.zeros([3260,10])

for i in range(0, len(Total_areas)):
    for k in range(0, (len(Total_areas.columns)-2)):
        j = k+1
        Total_areas_aggregate[i][k] = Total_areas.iloc[i][k]+Total_areas.iloc[i][j]
        
Total_areas_aggregate = pd.DataFrame(data=Total_areas_aggregate, 
                                     columns=['5', '15', '25', '35', '45', '55', '65', '75', '85', '95'])

Total_areas_aggregate['Class'] = Total_areas_aggregate.idxmax(axis=1)

#Number of classes
print('Total number of classes identified:', len(Total_areas_aggregate['Class'].unique()))
print('Classes are:', Total_areas_aggregate['Class'].unique())
Total_areas_aggregate.head()

Total number of classes identified: 6
Classes are: ['65' '55' '45' '75' '35' '25']


,5,15,25,35,45,55,65,75,85,95,Class
0,0.0,0.0,0.000000,0.457094,10.426908,40.215610,58.344114,53.471324,31.220354,5.855972,65
1,0.0,0.0,1.026304,8.978008,33.954291,50.797758,42.483829,27.072014,22.457956,13.152221,55
2,0.0,0.0,0.000000,0.871065,10.245796,38.792583,57.567917,53.134972,32.169038,7.201380,65
3,0.0,0.0,0.551962,8.762398,33.540319,51.901682,44.208711,26.321690,21.604140,13.014230,55
4,0.0,0.0,0.000000,0.448469,9.840448,39.163433,60.000000,54.920224,30.116429,5.467874,65


In [13]:
#Save classes into files for later use
np.save('D:\Projects\Research\SolarStudy\Classes.npy', Total_areas['Class'])
np.save('D:\Projects\Research\SolarStudy\AggregateClasses.npy', Total_areas_aggregate['Class'])

In [14]:
#Read all model parameter file addresses
path = r'D:\Projects\Research\SolarStudy\ParameterOutputs'                     
all_files = glob.glob(os.path.join(path, "*.xlsx")) # OS independent

In [25]:
#Extract GH input parameters (seed numbers) from the file names and save into a pandas dataframe
#Note: use these to run GH definition without the simulation and generate geometrical properties of 
#each model, e.g. heights, widths, distances, etc.
N_rows = len(all_files)
N_cols = 69 #number of model parameters from GH file

ModelParameters = pd.DataFrame(np.zeros((N_rows, N_cols)), columns=['(1,1) Height', '(1,2) Height', '(1,3) Height',
                                                           '(2,1) Height', '(1,2) Height', '(2,3) Height',
                                                           '(3,1) Height', '(3,2) Height', '(3,3) Height',
                                                           '(1,1) Xlength', '(1,2) Xlength', '(1,3) Xlength',
                                                           '(2,1) Xlength', '(1,2) Xlength', '(2,3) Xlength',
                                                           '(3,1) Xlength', '(3,2) Xlength', '(3,3) Xlength',
                                                           '(1,1) Ylength', '(1,2) Ylength', '(1,3) Ylength',
                                                           '(2,1) Ylength', '(1,2) Ylength', '(2,3) Ylength',
                                                           '(3,1) Ylength', '(3,2) Ylength', '(3,3) Ylength',
                                                           '(1,1) Rotation', '(1,2) Rotation', '(1,3) Rotation',
                                                           '(2,1) Rotation', '(1,2) Rotation', '(2,3) Rotation',
                                                           '(3,1) Rotation', '(3,2) Rotation', '(3,3) Rotation',
                                                           '(1,1)(1,2) MinDistance', '(1,2)(1,3) MinDistance',
                                                           '(1,1)(2,1) MinDistance', '(2,1)(1,2) MinDistance',
                                                           '(2,2)(2,3) MinDistance', '(1,3)(2,3) MinDistance',
                                                           '(2,1)(3,1) MinDistance', '(3,1)(3,2) MinDistance',
                                                           '(2,2)(3,2) MinDistance', '(3,2)(3,3) MinDistance',
                                                           '(3,3)(2,3) MinDistance',
                                                           '(1,1)(1,2)IncidentAngle', '(1,2)(1,1)IncidentAngle',
                                                           '(1,2)(1,3)IncidentAngle', '(1,3)(1,2)IncidentAngle',
                                                           '(1,1)(2,1)IncidentAngle', '(2,1)(1,1)IncidentAngle',
                                                           '(2,1)(2,2)IncidentAngle', '(2,2)(2,1)IncidentAngle',
                                                           '(2,2)(2,3)IncidentAngle', '(2,3)(2,2)IncidentAngle',
                                                           '(1,3)(2,3)IncidentAngle', '(2,3)(1,3)IncidentAngle',
                                                           '(2,1)(3,1)IncidentAngle', '(3,1)(2,1)IncidentAngle',
                                                           '(3,1)(3,2)IncidentAngle', '(3,2)(3,1)IncidentAngle',
                                                           '(2,2)(3,2)IncidentAngle', '(3,2)(2,2)IncidentAngle',
                                                           '(3,2)(3,3)IncidentAngle', '(3,3)(3,2)IncidentAngle',
                                                           '(3,3)(2,3)IncidentAngle', '(2,3)(3,3)IncidentAngle'])


model_data = np.empty([0,69])

for f in all_files:
    temp = pd.read_excel(f, header=None)
    model_data = np.append(model_data, temp, axis=0).astype('float32')

In [31]:
for i in range(0, N_rows):
    for j in range(0, N_cols):
        ModelParameters.iloc[i,j] = model_data[i][j]

ModelParameters.head()

,"(1,1) Height","(1,2) Height","(1,3) Height","(2,1) Height","(1,2) Height","(2,3) Height","(3,1) Height","(3,2) Height","(3,3) Height","(1,1) Xlength",...,"(2,1)(3,1)IncidentAngle","(3,1)(2,1)IncidentAngle","(3,1)(3,2)IncidentAngle","(3,2)(3,1)IncidentAngle","(2,2)(3,2)IncidentAngle","(3,2)(2,2)IncidentAngle","(3,2)(3,3)IncidentAngle","(3,3)(3,2)IncidentAngle","(3,3)(2,3)IncidentAngle","(2,3)(3,3)IncidentAngle"
0,69.0,44.0,57.0,51.0,44.0,38.0,32.0,26.0,20.0,24.0,...,55.418526,42.307568,40.416409,34.679089,54.811382,39.963696,35.804035,29.024542,50.496624,32.553974
1,69.0,44.0,57.0,51.0,44.0,38.0,32.0,26.0,20.0,24.0,...,55.656563,42.561646,43.969666,38.089035,53.833050,38.950439,37.613152,30.653898,53.920048,35.840199
2,69.0,44.0,57.0,51.0,44.0,38.0,32.0,26.0,20.0,24.0,...,55.656563,42.561646,43.969666,38.089035,53.833050,38.950439,37.613152,30.653898,53.920048,35.840199
3,69.0,44.0,57.0,51.0,44.0,38.0,32.0,26.0,20.0,24.0,...,56.482510,43.451180,42.060757,36.246628,54.326092,39.458744,35.188965,28.475880,49.131756,31.311167
4,69.0,44.0,57.0,51.0,44.0,38.0,32.0,26.0,20.0,24.0,...,56.482510,43.451180,42.060757,36.246628,54.326092,39.458744,35.188965,28.475880,49.131756,31.311167


In [165]:
np.save('D:\Projects\Research\SolarStudy\ModelParameters.npy', ModelParameters)